# **Quantum Walk Implementation on a Linear Graph Using Classiq Library**

### **Introduction**
Quantum walks, the quantum counterpart of classical random walks, are crucial for various quantum algorithms, such as those used in search and graph traversal. In this notebook, we will delve into the implementation of a quantum walk on a linear graph (a path graph) using the Classiq library. The focus will be on understanding the theoretical foundations and implementing the algorithm step-by-step.

### **Objectives**

1. **Comprehend the basics of quantum walks:** Explore the differences between quantum and classical walks and their significance in quantum computing.
2. **Implement the quantum walk algorithm on a linear graph:** Use Python and the Classiq library to simulate a quantum walk, focusing on the preparation, evolution, and measurement of quantum states.
3. **Visualize the quantum circuit:** Generate a quantum circuit that represents the quantum walk, providing a clear view of the operations performed on qubits.

## Background: Quantum Walks on Graphs
### Classical vs. Quantum Walks

- **Classical Random Walks:** In a classical random walk, a walker moves step-by-step on a graph, where each step is determined probabilistically. The walker might, for example, move to an adjacent vertex with equal probability.
- **Quantum Walks:** Quantum walks leverage quantum superposition and interference, allowing the quantum walker to explore multiple paths simultaneously. This parallelism can lead to significant speedups in quantum algorithms.

### The Quantum Walk Operators

In the context of quantum walks on a linear graph, we primarily focus on two operators:

1. Coin Operator (C Operator): This operator determines the direction of the quantum walk. It acts on the state ∣𝑗,𝑘⟩ where 𝑗 represents the current vertex and 𝑘 represents the adjacent vertex. The C operator is defined as:

$$C = \sum_{j \in V} |j\rangle\langle j| \otimes (2|\partial j\rangle\langle \partial j| - I)$$

Here, ∣∂𝑗⟩ is a superposition of the neighboring vertices of 𝑗, and 𝐼 is the identity operator.

2. **Shift Operator (S Operator):** This operator shifts the quantum state between adjacent vertices on the graph, effectively moving the quantum walker.

### Implementation of the Quantum Walk
We will now implement the quantum walk on a linear graph with 16 nodes using the Classiq library. The implementation will be broken down into several key steps, including the state preparation, definition of operators, and execution of the quantum walk.

### 1. Setup and Import Libraries

In [ ]:
!pip install -U classiq

In [ ]:
import classiq
classiq.authenticate()

In [12]:
# Import necessary libraries and modules
from classiq import *
from classiq.qmod.symbolic import logical_or
from classiq.execution import ExecutionPreferences

### 2. State Preparation: Creating the Initial State
The first step is to prepare the initial quantum state using the Hadamard gate to create a superposition over all possible vertex states.

In [2]:
@qfunc
def prepare_minus(x: QBit):
    """
    Prepare a quantum state |-> by applying an X gate followed by an H gate.
    This state is essential for the initial quantum walk configuration.
    """
    X(x)  # Flip the qubit state
    H(x)  # Create a superposition state |-> = (|0> - |1>)/sqrt(2)

**Explanation:**

- **X Gate:** The NOT gate flips the qubit from ∣0⟩ to ∣1⟩
.
**Hadamard Gate (H Gate):v This gate creates a superposition of ∣0⟩ and ∣1⟩
, resulting in the ∣−⟩ state.
 
### 3. Diffuser Oracle: Marking States of Interest
The diffuser oracle flips the phase of the state if a certain condition (e.g., 𝑥≠0) is met.

In [3]:
@qfunc
def diffuzer_oracle(aux: Output[QNum], x: QNum):
    """
    Oracle function that flips the auxiliary qubit if x != 0.
    This operation is used to mark states that need to be amplified in the quantum walk.
    """
    aux ^= (x != 0)  # Conditional phase flip of the auxiliary qubit

**Explanation:**

- **Oracle Function:** This marks states in the quantum superposition that fulfill certain conditions, a technique commonly used in quantum search algorithms.

### 4. Zero Diffuser: Amplitude Amplification
This function applies amplitude amplification, a crucial step in quantum algorithms to increase the probability of finding the correct solution.

In [4]:
@qfunc
def zero_diffuzer(x: QNum):
    """
    Creates a zero diffuser using auxiliary qubits.
    This operation amplifies the probability of measuring the correct state.
    """
    aux = QNum('aux')
    allocate(1, aux)
    within_apply(compute=lambda: prepare_minus(aux), 
                 action=lambda: diffuzer_oracle(aux, x))

**Explanation:**

- **Amplitude Amplification:** This technique amplifies the probability of the "marked" states, increasing the likelihood of measuring a desired outcome.

### 5. C-Iteration: Evolving the Quantum State with Coin Operator
The C operator is applied to simulate the quantum walk by determining the evolution of the quantum state across the graph.

In [5]:
def C_iteration(i: int, vertices: QNum, adjacent_vertices: QNum):
    """
    Implements a single iteration of the C operator, simulating the evolution of the quantum state.
    """
    num_nodes = 2**4
    prob = [0] * num_nodes
    if i == 0:
        prob[i + 1] = 1.0
    elif i == num_nodes - 1:
        prob[i - 1] = 1.0
    else:
        prob[i - 1] = 0.5
        prob[i + 1] = 0.5
    print(f'Node={i}, prob vec ={prob}')
    control(ctrl=vertices == i,
            operand=lambda: within_apply(
                compute=lambda: inplace_prepare_state(probabilities=prob, bound=0.01, target=adjacent_vertices),
                action=lambda: zero_diffuzer(adjacent_vertices)))

**Explanation:**

- **Probability Vector:** This vector defines the probability distribution for transitioning from one vertex to its neighbors. The quantum state evolves according to this distribution.

- **Control Condition:** The quantum state evolves based on the control condition, ensuring that the correct probability distribution is applied.

### 6. C Operator: Applying the Coin Operator
The C operator applies the evolution step across all vertices, effectively performing a quantum walk.

In [6]:
@qfunc
def C_operator(vertices: QNum, adjacent_vertices: QNum):
    """
    C operator that applies the coin operation across all vertex states.
    This operator is responsible for evolving the quantum state across the graph.
    """
    num_nodes = 2**4
    for i in range(num_nodes):
        C_iteration(i, vertices, adjacent_vertices)

**Explanation:**

- **C Operator:** This operator applies the coin operation, which determines the direction and evolution of the quantum walk by adjusting the quantum state based on the graph's structure.

### 7. Edge Oracle: Identifying Adjacent Vertices
The edge oracle identifies if two vertices are adjacent, a necessary step before shifting the quantum state.

In [7]:
@qfunc
def edge_oracle(res: Output[QBit], vertices: QNum, adjacent_vertices: QNum):
    """
    Edge oracle that identifies whether two vertices are adjacent.
    The oracle checks the condition |j - k| = 1, indicating adjacency.
    """
    res |= (((vertices - adjacent_vertices) == 1) | ((vertices - adjacent_vertices) == -1))

**Explanation:**

- **Edge Oracle:** This oracle checks for adjacency between vertices, crucial for implementing the shift operator in the quantum walk.

### 8. Bitwise Swap: Exchanging Qubit States
This function swaps the states of two quantum registers bit by bit, which is essential for implementing the shift operation in the quantum walk.

In [8]:
@qfunc 
def bitwise_swap(x: QArray[QBit], y: QArray[QBit]):
    """
    Swaps the bits of two quantum arrays.
    This operation is necessary for the S operator, which simulates movement in the quantum walk.
    """
    repeat(count=x.len,
           iteration=lambda i: SWAP(x[i], y[i]))

**Explanation:**

- **SWAP Gate:** This gate exchanges the states of two qubits, simulating the shift in the quantum walk.

### 9. S Operator: Implementing the Shift
The S operator shifts the quantum state between adjacent vertices, simulating the movement of the quantum walker on the graph.

In [9]:
@qfunc 
def S_operator(vertices: QNum, adjacent_vertices: QNum):
    """
    S operator that shifts the quantum state between adjacent vertices.
    This operator is key to moving the quantum walker along the graph.
    """
    res = QNum('res')
    edge_oracle(res, vertices, adjacent_vertices)  # Apply edge oracle to check adjacency
    control(ctrl=res == 1,
            operand=lambda: bitwise_swap(vertices, adjacent_vertices))  # Conditional bitwise swap

**Explanation:**

- **Shift Operation:** The S operator implements the physical movement of the quantum state along the graph by swapping the states of adjacent vertices.

### 10. Main Quantum Walk Function
Finally, we combine all the components to perform a quantum walk on the graph.

In [10]:
@qfunc 
def main(vertices: Output[QNum], adjacent_vertices: Output[QNum]):
    """
    Main function for the quantum walk algorithm.
    This function initializes the quantum states and applies the C and S operators to perform the quantum walk.
    """
    size = 4
    allocate(size, vertices)  # Allocate qubits for vertices
    hadamard_transform(vertices)  # Apply Hadamard transform to initialize superposition
    allocate(size, adjacent_vertices)  # Allocate qubits for adjacent vertices

    C_operator(vertices, adjacent_vertices)  # Apply coin operator
    S_operator(vertices, adjacent_vertices)  # Apply shift operator

**Explanation:**

- **Initialization:** The vertices are initialized in superposition using the Hadamard transform, setting up the quantum state for exploration.

- **Operators Application:** The C and S operators are applied sequentially to perform the quantum walk on the graph.

### 11. Synthesizing and Visualizing the Quantum Program
Finally, we synthesize the quantum circuit and visualize it using the Classiq library.

In [ ]:
# Synthesize and display the quantum program
qmod = create_model(main)
qprog = synthesize(qmod)
show(qprog)

**Explanation:**

- **Synthesis:** The high-level quantum algorithm is translated into a quantum circuit, ready for execution on a quantum computer.
- **Visualization:** The resulting quantum circuit is displayed, illustrating the sequence of operations performed on the qubits.

### Conclusion
In this notebook, we've implemented a quantum walk on a linear graph using the Classiq library. By carefully constructing and applying the C and S operators, we've simulated the evolution of a quantum state across the graph, demonstrating the principles of quantum walks. This implementation provides a foundation for understanding more complex quantum algorithms that rely on quantum walks.

### Key Takeaways:
- **Quantum Walks:** A powerful tool in quantum computing, offering parallelism and potential speedups for various algorithms.
- **Classiq Library:** A versatile tool for constructing and synthesizing quantum circuits, enabling the practical implementation of complex quantum algorithms.
- **Graph Theory in Quantum Computing:** The use of graphs and graph-based algorithms is central to many quantum computing applications, making this a critical area of study.

Feel free to experiment further with different graph structures or modify the quantum operations to explore their effects on the quantum walk. Let me know if you have any further questions or if there's anything else you'd like to explore!

